### 📘 1. Imports y configuración de entorno

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import mlflow
from tensorflow.keras.callbacks import EarlyStopping
from mlflow.models.signature import infer_signature

### 📘 2. Cargar los datos
En esta sección se cargan y dividen los datos preprocesados del pipeline anterior. Este paso es esencial para separar los conjuntos de entrenamiento y validación.

In [25]:

# Cargar datos procesados y divididos
data = np.load("../data/processed_split.npz")

X_train = data["X_train"]
X_val   = data["X_val"]
y_train = data["y_train"]
y_val   = data["y_val"]


### 📘 3. Arquitectura del Modelo
Aquí se define una arquitectura LSTM bidireccional, que mejora el desempeño al considerar tanto contexto anterior como posterior en el análisis de sentimiento.

In [26]:
embedding_dim = 64
lstm_units = 64
batch_size = 32
epochs = 20


### 🧠 4. Crear modelo (LSTM)
Se usan callbacks para evitar sobreajuste (`EarlyStopping`) y guardar automáticamente el mejor modelo basado en la métrica de validación (`ModelCheckpoint`).

In [28]:
# EarlyStopping callback
early_stop = EarlyStopping(
    monitor='val_accuracy',     
    patience=2,
    restore_best_weights=True,
    verbose=1
)

# Guardar el mejor modelo durante el entrenamiento
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1,
    save_format='tf'  # Guardar en formato TensorFlow
)

model = keras.Sequential([
    layers.Embedding(input_dim=20000, output_dim=embedding_dim, input_length=X_train.shape[1]),
    layers.Bidirectional(layers.LSTM(lstm_units)),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 64)           1280000   
                                                                 
 bidirectional_6 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1346177 (5.14 MB)
Trainable params: 1346177 (5.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 📊 5. Entrenamiento + MLflow tracking

In [29]:
# MLflow tracking
mlflow.set_experiment("IMDb Sentiment DL")

with mlflow.start_run():
    # Registrar hiperparámetros
    mlflow.log_param("embedding_dim", embedding_dim)
    mlflow.log_param("lstm_units", lstm_units)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)

    # Entrenar el modelo
    history = model.fit(X_train, y_train, 
                        validation_data=(X_val, y_val),
                        epochs=epochs, 
                        batch_size=batch_size,
                        callbacks=[early_stop, checkpoint]
                        )

    # Registrar métricas
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    mlflow.log_metric("val_loss", val_loss)
    mlflow.log_metric("val_accuracy", val_accuracy)

    #Registrar epocas real y final (early stop)
    final_epoch = len(history.history['loss'])
    mlflow.log_param("epochs_trained", final_epoch)

          
    # Guardar localmente (opcional)
    model.save("../models/sentiment_model.keras")

    # Inferir la signature
    signature = infer_signature(X_val, model.predict(X_val))
    input_example = X_val[:1]

    # Loguear en MLflow
    mlflow.keras.log_model(
        model=tf.keras.models.load_model("best_model"),
        artifact_path="sentiment_model",
        signature=signature,
        input_example=input_example
    )

Epoch 1/20
2500/2500 [==============================] - ETA: 0s - loss: 0.2902 - accuracy: 0.8783
Epoch 1: val_accuracy improved from -inf to 0.90125, saving model to best_model
INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


2500/2500 [==============================] - 418s 166ms/step - loss: 0.2902 - accuracy: 0.8783 - val_loss: 0.2405 - val_accuracy: 0.9013
Epoch 2/20
2500/2500 [==============================] - ETA: 0s - loss: 0.1939 - accuracy: 0.9260
Epoch 2: val_accuracy improved from 0.90125 to 0.91695, saving model to best_model
INFO:tensorflow:Assets written to: best_model/assets


INFO:tensorflow:Assets written to: best_model/assets


2500/2500 [==============================] - 416s 166ms/step - loss: 0.1939 - accuracy: 0.9260 - val_loss: 0.2188 - val_accuracy: 0.9169
Epoch 3/20
2500/2500 [==============================] - ETA: 0s - loss: 0.1488 - accuracy: 0.9457
Epoch 3: val_accuracy did not improve from 0.91695
2500/2500 [==============================] - 400s 160ms/step - loss: 0.1488 - accuracy: 0.9457 - val_loss: 0.2155 - val_accuracy: 0.9127
Epoch 4/20
2500/2500 [==============================] - ETA: 0s - loss: 0.1170 - accuracy: 0.9577Restoring model weights from the end of the best epoch: 2.

Epoch 4: val_accuracy did not improve from 0.91695
2500/2500 [==============================] - 398s 159ms/step - loss: 0.1170 - accuracy: 0.9577 - val_loss: 0.2481 - val_accuracy: 0.9140
Epoch 4: early stopping
625/625 [==============================] - 39s 60ms/step
INFO:tensorflow:Assets written to: /tmp/tmpntiyx3rp/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpntiyx3rp/model/data/model/assets
/home/alexor/ailab/project-2-nlp-sentiment/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 [==============================] - 1s 720ms/step
